In [93]:

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 0. Intro
1. Use InferNet to generate params values required by GT2.
2. Output those values for calculating (part of) input of MB-MDN.
# 1. Preparations
## 1.1 Global

In [102]:
import torch
import numpy as np
import random
import pandas as pd

from MLP.models import MLP_2_1
from importlib import reload

from MLP.my_collate_fn import my_collate_fn_InferNet
from torch.utils.data import DataLoader, SequentialSampler
from MLP.mydataset_GT2 import myDataset

from MLP.Config import config_GT_gen
reload(config_GT_gen)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

bound_alpha = torch.tensor([-0.3,0.3],device=device)
bound_labda = torch.tensor([0.01,18],device=device)

opt = config_GT_gen.DefaultConfig()

## 1.2 相关path

In [103]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

setup_seed(opt.seed)

dataset = myDataset(opt.train_path, opt.target_path_metric, opt.target_path_loss, opt.params_opitim_path,opt.data_key_path, opt.NLL_metric_path)


data_idx = np.arange(0,dataset.__len__()).tolist()
print("seed=",opt.seed)
print("arti=",opt.ARTIFICIAL)

seed= 4
arti= True


# 2. dataloader
1.

In [104]:
data_loader = DataLoader(dataset=dataset, batch_size=1, shuffle=False, num_workers=0, drop_last=False, sampler=SequentialSampler(data_idx), collate_fn=my_collate_fn_InferNet)

# 3. Load and Generate

In [105]:
if opt.ARTIFICIAL:
    mlp = MLP_2_1()
else:
    mlp = MLP_2_1()
model_data = torch.load(opt.model_params_MLP)
mlp.load_state_dict(model_data)
mlp = mlp.to(device=device)

In [106]:
params_arr = np.array([0.]*(dataset.__len__()*2)).reshape(-1,2)
params = pd.DataFrame(params_arr,columns = ['alpha','labda'])

In [107]:
model=mlp.eval()
for batch_id, data in enumerate(data_loader):

    input_data, _,_, _, _, _= data
    # Do the inference
    input_data = input_data.to(device)

    Alpha, Labda = mlp(input_data)

    Alpha = Alpha.detach().cpu().numpy()
    Labda = Labda.detach().cpu().numpy()

    params.loc[batch_id,'alpha'] = Alpha
    params.loc[batch_id,'labda'] = Labda

print("Done")
print(params.head())

Done
      alpha     labda
0  0.094390  0.746689
1  0.220394  0.953212
2  0.138472  0.961662
3  0.205837  1.214306
4  0.199331  0.885956


- save

In [108]:
print(opt.params_gen_path)

../../data/SA_PT/params_artificial_v2_noise=0.05_seed=4.csv


In [109]:
params.to_csv(opt.params_gen_path,encoding="utf-8", header=True,index=False)
